# Lesson Overview 

1. Review of TF Transform (TFT)

2. Dataset Review & Define Raw Features

3. Run steps from previous TF Data Validation & TF Transform pipeline

4. Define an Estimator 

5. Train, Evaluate, and Export our model

6. Construct an Apache Beam Pipeline

7. Define Data SliceSpecs for Model Analysis

8. Run TFMA

9. Show data sliced along feature column pickup_hour

10. show metrics sliced by COLUMN_CROSS_VALUE_SPEC 

11. Show overall metrics
 

## TensorFlow Transform

#### The Feature Engineering Component of TensorFlow Extended (TFX)

[TensorFlow Transform](https://github.com/tensorflow/transform) is a library for preprocessing data with TensorFlow. `tf.Transform` is useful for data that requires a full-pass, such as:

+ Normalize an input value by mean and standard deviation.

+ Convert strings to integers by generating a vocabulary over all input values.

+ Convert floats to integers by assigning them to buckets based on the observed data distribution.

+ TensorFlow has built-in support for manipulations on a single example or a batch of examples. `tf.Transform` extends these capabilities to support full-passes over the example data.

We will export the output of `tf.Transform` as a TensorFlow graph to use for training and serving. Using the same graph for both training and serving can prevent skew since the same transformations are applied in both stages.

## Dataset

The dataset will be using throughout this session will be the New York Yellow Cab dataset available via [BigQuery public datasets](https://console.cloud.google.com/marketplace/details/city-of-new-york/nyc-tlc-trips?filter=solution-type:dataset&filter=category:encyclopedic).

Here is an example of how to extract data:
```
SELECT vendor_id,
       EXTRACT(MONTH FROM pickup_datetime) AS pickup_month,
       EXTRACT(HOUR FROM pickup_datetime) AS pickup_hour,
       EXTRACT(DAYOFWEEK FROM pickup_datetime) AS pickup_day_of_week, 
       EXTRACT(MONTH FROM dropoff_datetime) AS dropoff_month,
       EXTRACT(HOUR FROM dropoff_datetime) AS dropoff_hour,
       EXTRACT(DAYOFWEEK FROM dropoff_datetime) AS dropoff_day_of_week,
       passenger_count,
       store_and_fwd_flag, 
       trip_distance,
       fare_amount,
       tip_amount,
       payment_type,
       trip_type
FROM `bigquery-public-data.new_york_taxi_trips.tlc_green_trips_2018`  

```

The columns in the dataset are:

## Install Dependencies

Third party dependencies can be found in `requirements.txt` and already have been installed. The specific versions matter, please look at [this](https://github.com/tensorflow/tfx#compatible-versions) for more info.

In [1]:
%%bash
pip install tensorflow==1.14.0
pip install tfx==0.14.0rc1
pip install apache-beam==2.14.0 
pip install tensorflow-data-validation==0.14.1
pip install tensorflow-metadata==0.14.0
pip install tensorflow-model-analysis==0.14.0
pip install tensorflow-transform==0.14.0


  Created wheel for tensorflow-transform: filename=tensorflow_transform-0.14.0-cp27-none-any.whl size=282795 sha256=d7c5a4d612c4592bb4479a88ff949ba808304b6688b658a1931b7f4a0cd829af
  Stored in directory: /root/.cache/pip/wheels/2f/8f/19/808f4a2d4d23a13b6ec44682fc2662646e8d9193b49f4a5f93
  Created wheel for avro: filename=avro-1.9.1-cp27-none-any.whl size=41131 sha256=be22d13cef2e2d0152f8e111a71024122323b852c853408f3df86cdc0d2f509c
  Stored in directory: /root/.cache/pip/wheels/fe/03/0c/354f2a3b1e10ab337b45728410509de62fbc0f7fe09ad196a2
  Created wheel for hdfs: filename=hdfs-2.5.8-cp27-none-any.whl size=33214 sha256=21a98c7cce8e90a50622d4bfd4d62ff28b12b744428f9ee3fe06618765979684
  Stored in directory: /root/.cache/pip/wheels/fe/a7/05/23e3699975fc20f8a30e00ac1e515ab8c61168e982abe4ce70
  Created wheel for dill: filename=dill-0.2.9-cp27-none-any.whl size=77405 sha256=3abc0b26d455b1c94271df8740f9b1ec94f29e53616c5c3ba7f1dc50aa82572f
  Stored in directory: /root/.cache/pip/wheels/5b/d7/0f/e

ERROR: fastai 0.7.0 has requirement torch<0.4, but you'll have torch 1.2.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
ERROR: multiprocess 0.70.8 has requirement dill>=0.3.0, but you'll have dill 0.2.9 which is incompatible.
ERROR: google-cloud-storage 1.16.1 has requirement google-cloud-core<2.0dev,>=1.0.0, but you'll have google-cloud-core 0.29.1 which is incompatible.
ERROR: google-cloud-translate 1.5.0 has requirement google-cloud-core<2.0dev,>=1.0.0, but you'll have google-cloud-core 0.29.1 which is incompatible.


## Load necessary packages

In [2]:
import warnings
warnings.filterwarnings("ignore")

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tempfile
import urllib
import tfx
from tfx.components.evaluator.component import Evaluator
from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
from tfx.components.example_validator.component import ExampleValidator
from tfx.components.model_validator.component import ModelValidator
from tfx.components.pusher.component import Pusher
from tfx.components.schema_gen.component import SchemaGen
from tfx.components.statistics_gen.component import StatisticsGen
from tfx.components.trainer.component import Trainer
from tfx.components.transform.component import Transform
from tfx.orchestration.interactive.interactive_context import InteractiveContext
from tfx.proto import evaluator_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto.evaluator_pb2 import SingleSlicingSpec
from tfx.utils.dsl_utils import csv_input

import os
import shutil
import argparse
import tensorflow as tf
import apache_beam as beam  
import tensorflow_transform as tft
from google.protobuf import text_format 
import tensorflow_model_analysis as tfma
import tensorflow_data_validation as tfdv
import tensorflow_transform.beam as tft_beam
from tensorflow.python.lib.io import file_io
from tensorflow_metadata.proto.v0 import schema_pb2
from tensorflow_transform import coders as tft_coders
from tensorflow_transform.tf_metadata import metadata_io
from tensorflow_transform.saved import saved_transform_io
from tensorflow_transform.beam.tft_beam_io import transform_fn_io
from tensorflow_transform.coders import example_proto_coder
from tensorflow_transform.tf_metadata import dataset_metadata
from tensorflow_transform.tf_metadata import dataset_schema
from tensorflow_transform.tf_metadata import schema_utils

print('TFDV version: {}'.format(tfdv.version.__version__))
print('TF version: {}'.format(tf.VERSION))

W1003 02:49:33.339711 140550488856448 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/tfx/components/transform/executor.py:57: The name tf.FixedLenFeature is deprecated. Please use tf.io.FixedLenFeature instead.

W1003 02:49:33.341072 140550488856448 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tfx/components/transform/executor.py:57: from_feature_spec (from tensorflow_transform.tf_metadata.dataset_schema) is deprecated and will be removed in a future version.
Instructions for updating:
from_feature_spec is a deprecated, use schema_utils.schema_from_feature_spec


TFDV version: 0.14.1
TF version: 1.14.0


## Download Data

The data we will be using for this lesson is available via Google Cloud Storage. Here's how to download it to your Colab instance. 

In [3]:
%%bash
mkdir -p /tmp/data/
mkdir -p /tmp/data/train
mkdir -p /tmp/data/serving
mkdir -p /tmp/data/eval
mkdir -p /tmp/data/module

wget -P /tmp/data/train/ https://storage.googleapis.com/tfx_course/data/train/train.csv 
wget -P /tmp/data/serving/ https://storage.googleapis.com/tfx_course/data/serving/serving.csv 
wget -P /tmp/data/eval/ https://storage.googleapis.com/tfx_course/data/eval/eval.csv 
wget -P /tmp/data/module https://storage.googleapis.com/tfx_course/taxi_utils.py

--2019-10-03 02:49:33--  https://storage.googleapis.com/tfx_course/data/train/train.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 2607:f8b0:400c:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 273804 (267K) [application/octet-stream]
Saving to: ‘/tmp/data/train/train.csv’

     0K .......... .......... .......... .......... .......... 18% 53.5M 0s
    50K .......... .......... .......... .......... .......... 37% 52.7M 0s
   100K .......... .......... .......... .......... .......... 56% 72.9M 0s
   150K .......... .......... .......... .......... .......... 74% 77.8M 0s
   200K .......... .......... .......... .......... .......... 93% 72.9M 0s
   250K .......... .......                                    100% 70.2M=0.004s

2019-10-03 02:49:33 (64.5 MB/s) - ‘/tmp/data/train/train.csv’ saved [273804/273804]

--2019-10-03 02:49:33--  https://s

## Define Lesson-wide Parameters

In [0]:
BASE_DIR = os.getcwd()

DATA_DIR = os.path.join(BASE_DIR, '/tmp/data')

OUTPUT_DIR = os.path.join(BASE_DIR)

_serving_model_dir = os.path.join(tempfile.mkdtemp(),
                                  'serving_model/taxi_simple')

# base dir containing train and eval data
TRAIN_DATA_DIR = os.path.join(DATA_DIR, 'train')
EVAL_DATA_DIR = os.path.join(DATA_DIR, 'eval')
SERVING_DATA_DIR = os.path.join(DATA_DIR, 'serving')

TRAIN_DATA = os.path.join(TRAIN_DATA_DIR, 'train.csv')
EVAL_DATA = os.path.join(EVAL_DATA_DIR, 'eval.csv')
SERVING_DATA = os.path.join(SERVING_DATA_DIR, 'serving.csv')

# base dir where TFT writes training data
TFT_TRAIN_OUTPUT_BASE_DIR = os.path.join(OUTPUT_DIR, 'tft_train')
TFT_TRAIN_FILE_PREFIX = 'train_transformed'

# base dir where TFT writes eval data
TFT_EVAL_OUTPUT_BASE_DIR = os.path.join(OUTPUT_DIR, 'tft_eval')
TFT_EVAL_FILE_PREFIX = 'eval_transformed'

TF_OUTPUT_BASE_DIR = os.path.join(OUTPUT_DIR, 'tf')
 
SERVING_MODEL_DIR = 'serving_model_dir'
EVAL_MODEL_DIR = 'eval_model_dir'

## Remove output from previous runs

In [0]:
shutil.rmtree(TFT_TRAIN_OUTPUT_BASE_DIR, ignore_errors=True)
shutil.rmtree(TFT_EVAL_OUTPUT_BASE_DIR, ignore_errors=True)
shutil.rmtree(TF_OUTPUT_BASE_DIR, ignore_errors=True)
ny_taxi_module = os.path.join("/tmp/data/module/", 'taxi_utils.py')

## Define Raw Features

In [0]:
# categorical features are assumed to each have a maximum value in the dataset.
MAX_CATEGORICAL_FEATURE_VALUES = [2]

CATEGORICAL_FEATURE_KEYS = []

DENSE_FLOAT_FEATURE_KEYS = ['trip_distance', 'passenger_count', 'tip_amount']

# number of buckets used by tf.transform for encoding each feature.
FEATURE_BUCKET_COUNT = 10

BUCKET_FEATURE_KEYS = ['pickup_hour', 
                       'pickup_month', 
                       'pickup_day_of_week', 
                       'dropoff_month',
                       'dropoff_hour',
                       'dropoff_day_of_week']

# number of vocabulary terms used for encoding VOCAB_FEATURES by tf.transform
VOCAB_SIZE = 1000

# count of out-of-vocab buckets in which unrecognized VOCAB_FEATURES are hashed.
OOV_SIZE = 10

VOCAB_FEATURE_KEYS = []

LABEL_KEY = 'fare_amount'

CSV_COLUMN_NAMES = [
    'vendor_id',
    'pickup_month',
    'pickup_hour',
    'pickup_day_of_week',
    'dropoff_month',
    'dropoff_hour',
    'dropoff_day_of_week',
    'passenger_count',
    'trip_distance',
    'fare_amount',
    'tip_amount',
    'payment_type',
    'trip_type',]

## Run steps from previous TF Data Validation pipeline

In [7]:
context = InteractiveContext()

examples = csv_input(TRAIN_DATA_DIR)

# ingest data into pipeline
example_gen = CsvExampleGen(input_base=examples)
context.run(example_gen)

statistics_gen = StatisticsGen(
    input_data=example_gen.outputs['examples'])
context.run(statistics_gen)

import tensorflow_data_validation as tfdv
from tfx.types.artifact_utils import get_split_uri

artifact_list = statistics_gen.outputs['output'].get()
train_artifact_uri = get_split_uri(artifact_list, 'train')
train_stats_path = os.path.join(train_artifact_uri, 'stats_tfrecord')

train_stats = tfdv.load_statistics(train_stats_path)
tfdv.visualize_statistics(train_stats)

W1003 02:49:34.062352 140550488856448 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/tfx/orchestration/component_launcher.py:87: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

W1003 02:49:34.131833 140550488856448 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/tfx/components/base/base_driver.py:44: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.

W1003 02:49:34.171750 140550488856448 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/tfx/components/example_gen/csv_example_gen/executor.py:83: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.

W1003 02:49:38.112341 140550488856448 tfrecordio.py:57] Couldn't find python-snappy so the implementation of _TFRecordUtil._masked_crc32c is not as fast as it could be.
W1003 02:49:47.482131 140550488856448 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow_dat

In [8]:
infer_schema = SchemaGen(
    stats=statistics_gen.outputs['output'],
    infer_feature_shape=False)
context.run(infer_schema)


schema_dir = infer_schema.outputs['output'].get()[0].uri
schema_path = os.path.join(schema_dir, 'schema.pbtxt')

schema = tfdv.load_schema_text(schema_path)
tfdv.display_schema(schema)

W1003 02:49:47.569653 140550488856448 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/tfx/utils/io_utils.py:76: The name tf.gfile.ListDirectory is deprecated. Please use tf.io.gfile.listdir instead.



,Type,Presence,Valency,Domain
Feature name,,,,
'payment_type',INT,required,single,-
'trip_distance',FLOAT,required,single,-
'pickup_day_of_week',INT,required,single,-
'fare_amount',FLOAT,required,single,-
'trip_type',INT,required,single,-
'dropoff_day_of_week',INT,required,single,-
'dropoff_hour',INT,required,single,-
'pickup_hour',INT,required,single,-
'vendor_id',INT,required,single,-


In [9]:
schema

feature {
  name: "payment_type"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "trip_distance"
  value_count {
    min: 1
    max: 1
  }
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "pickup_day_of_week"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "fare_amount"
  value_count {
    min: 1
    max: 1
  }
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "trip_type"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "dropoff_day_of_week"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "dropoff_hour"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction

In [10]:
validate_stats = ExampleValidator(
    stats=statistics_gen.outputs['output'],
    schema=infer_schema.outputs['output'])
context.run(validate_stats)

validation_dir = validate_stats.outputs['output'].get()[0].uri
anomalies_path = os.path.join(validation_dir, 'anomalies.pbtxt')


anomalies = tfdv.load_anomalies_text(anomalies_path)
tfdv.display_anomalies(anomalies)

**Transform TFX Pipeline Component**

Rather than writing user defined functions to perform feature engineering, the benefit of using tf.Transform is that the transforms become part of the Tensorflow Graph. This eliminates training vs serving skew. The TFX Pipeline Component interface is as follows...

* Input: Data formatted in tf.Examples emitted from ExampleGen using schema generated by SchemaGen

* Output: SavedModel to a Trainer component

To start off, you'll need to define a `preprocessing_fn` method inside a python module which is then passed it as a parameter to Transform component. TFX Transform will find a method called `preprocessing_fn` and use it to construct the preprocessing pipeline.

`tf.Transform` helps us prevent train/serve skew by using the same code for both training and serving.

In order to prevent train/serve skew, we need to train a model & prepare our trained model for production by creating input functions. The main difference between our training input function and our serving input function is that training data contains the labels, and production data does not.  

Here is an example to demonstrate sample usage...






In [11]:
transform = Transform(
    input_data=example_gen.outputs['examples'],
    schema=infer_schema.outputs['output'],
    module_file=ny_taxi_module)
context.run(transform)

W1003 02:49:47.853775 140550488856448 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/tfx/components/transform/executor.py:241: The name tf.logging.debug is deprecated. Please use tf.compat.v1.logging.debug instead.

W1003 02:49:47.860295 140550488856448 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tfx/components/transform/executor.py:406: Schema (from tensorflow_transform.tf_metadata.dataset_schema) is deprecated and will be removed in a future version.
Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`
W1003 02:49:47.914194 140550488856448 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow_transform/mappers.py:349: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


inputs:  {u'trip_distance': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fd42b7b8d90>, u'pickup_day_of_week': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fd3f1a86050>, u'vendor_id': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fd3f3c837d0>, u'tip_amount': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fd401370c50>, u'pickup_month': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fd406812bd0>, u'fare_amount': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fd42a1dc450>, u'pickup_hour': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fd406812490>, u'dropoff_hour': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fd42b7e2b50>, u'dropoff_month': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fd47241e550>, u'passenger_count': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7

W1003 02:49:48.654891 140550488856448 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/saved_model/signature_def_utils_impl.py:201: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


inputs:  {u'trip_distance': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fd3f4d9e410>, u'pickup_day_of_week': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fd3f7fc1f10>, u'vendor_id': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fd3fa1a4f50>, u'tip_amount': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fd3f7fb35d0>, u'dropoff_hour': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fd3f7fb3f10>, u'fare_amount': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fd3fa18e5d0>, u'pickup_hour': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fd3fa1a4b10>, u'pickup_month': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fd3f7fb3b10>, u'dropoff_month': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fd3fa1a45d0>, u'passenger_count': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7

W1003 02:50:05.708560 140550488856448 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/tfx/utils/io_utils.py:87: The name tf.gfile.IsDirectory is deprecated. Please use tf.io.gfile.isdir instead.

W1003 02:50:05.710045 140550488856448 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/tfx/utils/io_utils.py:88: The name tf.gfile.DeleteRecursively is deprecated. Please use tf.io.gfile.rmtree instead.



ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_output: Channel(
            type_name: TransformPath
            artifacts: [Artifact(type_name: TransformPath, uri: /tmp/tfx-interactive-2019-10-03T02_49_34.059373-grhdOa/Transform/transform_output/5/, split: , id: 8)]
        )
        transformed_examples: Channel(
            type_name: ExamplesPath
            artifacts: [Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-10-03T02_49_34.059373-grhdOa/Transform/transformed_examples/5/train/, split: train, id: 9)
            Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-10-03T02_49_34.059373-grhdOa/Transform/transformed_examples/5/eval/, split: eval, id: 10)]
        ))

`ny_taxi_module` contains the following `preprocessing_fn` method:

In [0]:
def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.

  Args:
    inputs: map from feature keys to raw not-yet-transformed features.

  Returns:
    Map from string feature key to transformed feature operations.
  """
  outputs = {}
  for key in _DENSE_FLOAT_FEATURE_KEYS:
    # Preserve this feature as a dense float, setting nan's to the mean.
    outputs[_transformed_name(key)] = tft.scale_to_z_score(
        _fill_in_missing(inputs[key]))

  for key in _VOCAB_FEATURE_KEYS:
    # Build a vocabulary for this feature.
    outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(
        _fill_in_missing(inputs[key]),
        top_k=_VOCAB_SIZE,
        num_oov_buckets=_OOV_SIZE)

  for key in _BUCKET_FEATURE_KEYS:
    outputs[_transformed_name(key)] = tft.bucketize(
        _fill_in_missing(inputs[key]), _FEATURE_BUCKET_COUNT)

  for key in _CATEGORICAL_FEATURE_KEYS:
    outputs[_transformed_name(key)] = _fill_in_missing(inputs[key])

  fare_amount = _fill_in_missing(inputs[_LABEL_KEY])

  outputs[_transformed_name(_LABEL_KEY)] = fare_amount
  return outputs

**TFX Transform Component includes Feature Column Transformations:**

*   **Embedding**: converting sparse features (like the integer IDs produced by a vocabulary) into dense features by finding a meaningful mapping from high- dimensional space to low dimensional space. 

*   **Vocabulary generation**: converting strings or other non-numeric features into integers by creating a vocabulary that maps each unique value to an ID number.
*   **Normalizing values**: transforming numeric features so that they all fall within a similar range.


*   **Bucketization**: converting continuous-valued features into categorical features by assigning values to discrete buckets.


*   **Enriching text features**: producing features from raw data like tokens, n-grams, entities, sentiment, etc., to enrich the feature set.

Highly recommend you look at the documetation for [FeatureColumns](https://www.tensorflow.org/guide/feature_columns) as it will save you development time. TFX Transform transformations are implemented using Apache Beam & require a full pass over your dataset.










## Define an Estimator to Train a Model




```
def _build_estimator(config, hidden_units=None, warm_start_from=None):
  """Build an estimator for predicting the tipping behavior of taxi riders.

  Args:
    config: tf.estimator.RunConfig defining the runtime environment for the
      estimator (including model_dir).
    hidden_units: [int], the layer sizes of the DNN (input layer first)
    warm_start_from: Optional directory to warm start from.

  Returns:
    A dict of the following:
      - estimator: The estimator that will be used for training and eval.
      - train_spec: Spec for training.
      - eval_spec: Spec for eval.
      - eval_input_receiver_fn: Input function for eval.
  """
  real_valued_columns = [
      tf.feature_column.numeric_column(key, shape=())
      for key in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
  ]
  categorical_columns = [
      tf.feature_column.categorical_column_with_identity(
          key, num_buckets=_VOCAB_SIZE + _OOV_SIZE, default_value=0)
      for key in _transformed_names(_VOCAB_FEATURE_KEYS)
  ]
  categorical_columns += [
      tf.feature_column.categorical_column_with_identity(
          key, num_buckets=_FEATURE_BUCKET_COUNT, default_value=0)
      for key in _transformed_names(_BUCKET_FEATURE_KEYS)
  ]
  categorical_columns += [
      tf.feature_column.categorical_column_with_identity(  # pylint: disable=g-complex-comprehension
          key,
          num_buckets=num_buckets,
          default_value=0) for key, num_buckets in zip(
              _transformed_names(_CATEGORICAL_FEATURE_KEYS),
              _MAX_CATEGORICAL_FEATURE_VALUES)
  ]
  return tf.estimator.DNNLinearCombinedRegressor(
      config=config,
      # linear_feature_columns=categorical_columns,
      dnn_feature_columns=real_valued_columns,
      dnn_hidden_units=hidden_units or [100, 70, 50, 25],
      warm_start_from=warm_start_from)
```



In [13]:
trainer = Trainer(
    module_file=ny_taxi_module,
    transformed_examples=transform.outputs['transformed_examples'],
    schema=infer_schema.outputs['output'],
    transform_output=transform.outputs['transform_output'],
    train_args=trainer_pb2.TrainArgs(num_steps=10000),
    eval_args=trainer_pb2.EvalArgs(num_steps=5000))
context.run(trainer)

W1003 02:50:05.896872 140550488856448 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/training/training_util.py:236: initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W1003 02:50:05.920994 140550488856448 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/data/experimental/ops/readers.py:835: parallel_interleave (from tensorflow.python.data.experimental.ops.interleave_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.
W1003 02:50:05.947951 140550488856448 deprecation.py:

ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        output: Channel(
            type_name: ModelExportPath
            artifacts: [Artifact(type_name: ModelExportPath, uri: /tmp/tfx-interactive-2019-10-03T02_49_34.059373-grhdOa/Trainer/output/6/, split: , id: 11)]
        ))

## Define Data SliceSpecs for Model Analysis

In [0]:
# an empty slice spec means the whole dataset
OVERALL_SLICE_SPEC = evaluator_pb2.SingleSlicingSpec()

# data can be sliced along a feature column. 
FEATURE_COLUMN_SLICE_SPEC = evaluator_pb2.SingleSlicingSpec(
    column_for_slicing=['pickup_hour'])
        
# slices are computed for pickup_day_of_week x pickup_month.
FEATURE_COLUMN_CROSS_SPEC = evaluator_pb2.SingleSlicingSpec(
    column_for_slicing=['trip_distance', 'tip_amount'])

ALL_SPECS = [
    OVERALL_SLICE_SPEC,
    FEATURE_COLUMN_SLICE_SPEC,
    FEATURE_COLUMN_CROSS_SPEC,
]

## Visualize features of our trained model

In [15]:
model_analyzer = Evaluator(
    examples=example_gen.outputs['examples'],
    model_exports=trainer.outputs['output'],
    feature_slicing_spec=evaluator_pb2.FeatureSlicingSpec(
        specs=ALL_SPECS
    ))
context.run(model_analyzer)

W1003 02:50:40.268747 140550488856448 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow_model_analysis/eval_saved_model/load.py:163: load (from tensorflow.python.saved_model.loader_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
W1003 02:50:40.526190 140550488856448 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow_model_analysis/eval_saved_model/graph_ref.py:189: get_tensor_from_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.get_tensor_from_tensor_info or tf.compat.v1.saved_model

ExecutionResult(
    component_id: Evaluator
    execution_id: 7
    outputs:
        output: Channel(
            type_name: ModelEvalPath
            artifacts: [Artifact(type_name: ModelEvalPath, uri: /tmp/tfx-interactive-2019-10-03T02_49_34.059373-grhdOa/Evaluator/output/7/, split: , id: 12)]
        ))

In [0]:
PATH_TO_RESULT = model_analyzer.outputs['output'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)


## Show data sliced along feature column pickup_hour


In [23]:
tfma.view.render_slicing_metrics(tfma_result, slicing_column='pickup_hour')

## Show overall metrics

In [18]:
tfma.view.render_slicing_metrics(tfma_result)

## Model Validation

In [19]:
model_validator = ModelValidator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['output'])
context.run(model_validator)

ExecutionResult(
    component_id: ModelValidator
    execution_id: 8
    outputs:
        blessing: Channel(
            type_name: ModelBlessingPath
            artifacts: [Artifact(type_name: ModelBlessingPath, uri: /tmp/tfx-interactive-2019-10-03T02_49_34.059373-grhdOa/ModelValidator/blessing/8/, split: , id: 13)]
        ))

## Pusher

In [20]:
pusher = Pusher(
    model_export=trainer.outputs['output'],
    model_blessing=model_validator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 9
    outputs:
        model_push: Channel(
            type_name: ModelPushPath
            artifacts: [Artifact(type_name: ModelPushPath, uri: /tmp/tfx-interactive-2019-10-03T02_49_34.059373-grhdOa/Pusher/model_push/9/, split: , id: 14)]
        ))

In [0]:
SERVING_MODEL_DIR = 'serving_model_dir'
EVAL_MODEL_DIR = 'eval_model_dir'

TRAIN_BATCH_SIZE = 40
EVAL_BATCH_SIZE = 40

FIRST_DNN_LAYER_SIZE = 100
NUM_DNN_LAYERS = 4
DNN_DECAY_FACTOR = 0.7

## Summary

+ We used `tfma` to visualize how the model performs both over slice(s) of data as well as the on the overall dataset. 

+ We created an input function that we can be used when we deploy our trained model in a production environment to perform inference. As a result, we will avoid any issues pertaining to data skew since the same code is used during training & inference.  

In [22]:
_serving_model_dir

'/tmp/tmpaYd6vl/serving_model/taxi_simple'

## Reference

The example above leverages code from the following [source](https://github.com/tensorflow/tfx/tree/master/tfx/examples/chicago_taxi_pipeline).